# Calculate air quality statistics example

The World Health Organization (WHO) has carried out extensive research to link air concentrations
to health. To assess the air quality of a given territory, the hourly measurements need to be
compared to the following values:

In [24]:
#Long term
WHO_AQG_long_term = [{
    'pollutant': 'pm25', 'id': 9, 'AQG_val': 5, 'IT4': 10, 'IT3': 15, 'IT2': 25, 'IT1': 35}, {
    'pollutant': 'pm10', 'id': 10, 'AQG_val': 15, 'IT4': 20, 'IT3': 30, 'IT2': 50, 'IT1': 70}, {
    'pollutant': 'o3', 'id': None, 'AQG_val': 60, 'IT4': None, 'IT3': None, 'IT2': 70, 'IT1': 100}, {
    'pollutant': 'no2', 'id': 8, 'AQG_val': 10, 'IT4': None, 'IT3': 20, 'IT2': 30, 'IT1': 40}, {
 }]

## Belgium data
This was obtained from the EEA's webpage.

### Data gathering

In [1]:
# data preparation and analysis 
import numpy as np
import pandas as pd
import os
import glob
import valid_hours
import airqstats

In [2]:
# data path
path_single = 'Data/Belgium/Belgium_hourly_2022/CO_BELGIUM/BE_10_40505_2022_timeseries.csv'
path_all = 'Data/Belgium/Belgium_hourly_2022'

In [3]:
# inspection of single file to determine columns to import
test_be = pd.read_csv(path_single)
test_be

,Countrycode,Namespace,AirQualityNetwork,AirQualityStation,AirQualityStationEoICode,SamplingPoint,SamplingProcess,Sample,AirPollutant,AirPollutantCode,AveragingTime,Concentration,UnitOfMeasurement,DatetimeBegin,DatetimeEnd,Validity,Verification
0,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.135,mg/m3,2022-01-01 01:00:00 +01:00,2022-01-01 02:00:00 +01:00,1,3
1,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.135,mg/m3,2022-01-01 02:00:00 +01:00,2022-01-01 03:00:00 +01:00,1,3
2,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.135,mg/m3,2022-01-01 03:00:00 +01:00,2022-01-01 04:00:00 +01:00,1,3
3,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.130,mg/m3,2022-01-01 04:00:00 +01:00,2022-01-01 05:00:00 +01:00,1,3
4,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.140,mg/m3,2022-01-01 05:00:00 +01:00,2022-01-01 06:00:00 +01:00,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.110,mg/m3,2022-12-30 21:00:00 +01:00,2022-12-30 22:00:00 +01:00,1,3
8588,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,NaN,mg/m3,2022-12-29 00:00:00 +01:00,2022-12-29 01:00:00 +01:00,-1,3
8589,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,0.110,mg/m3,2022-12-30 22:00:00 +01:00,2022-12-30 23:00:00 +01:00,1,3
8590,BE,BE.CELINE-IRCEL.AQ,NET-Wallonia,STA-BETN085,BETN085,SPO-BETN085_00010_100,SPP-BETN085_00010_1,NaN,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,NaN,mg/m3,2022-12-30 23:00:00 +01:00,2022-12-31 00:00:00 +01:00,-1,3


### Data assessment

##### Wrangling required 
1. **Column selection**  
Several columns are not relevant for this study for now. We need to consider if nominal information of the location is neccesary.
 
2. **Data Preparation**  
Each pollutant had their own df; looping is done using stations grouping them yearly.

3. **Date columns are str**
Data types need to be changed.

In [4]:
cols2drop = ['Namespace', 'AirQualityStation', 'SamplingPoint', 'SamplingProcess', 'Sample', 'AirPollutantCode']

In [5]:
# Merge files for all pollutants
os.path.exists(path_all)
pollutants_files = []

for subdir, dirs, files in os.walk(path_all):
    for file in files:
        fp = glob.glob(os.path.join(subdir, file))
        pollutants_files.append(fp[0])
df_bg_allpollutants = pd.concat([pd.read_csv(fp, on_bad_lines = 'skip').drop(columns = cols2drop) for fp in pollutants_files])
df_bg_allpollutants

,Countrycode,AirQualityNetwork,AirQualityStationEoICode,AirPollutant,AveragingTime,Concentration,UnitOfMeasurement,DatetimeBegin,DatetimeEnd,Validity,Verification
0,BE,NET-Flanders,BETN029,O3,hour,41.0,µg/m3,2022-01-01 01:00:00 +01:00,2022-01-01 02:00:00 +01:00,1,3
1,BE,NET-Flanders,BETN029,O3,hour,40.0,µg/m3,2022-01-01 03:00:00 +01:00,2022-01-01 04:00:00 +01:00,1,3
2,BE,NET-Flanders,BETN029,O3,hour,34.5,µg/m3,2022-01-01 04:00:00 +01:00,2022-01-01 05:00:00 +01:00,1,3
3,BE,NET-Flanders,BETN029,O3,hour,31.5,µg/m3,2022-01-01 05:00:00 +01:00,2022-01-01 06:00:00 +01:00,1,3
4,BE,NET-Flanders,BETN029,O3,hour,35.0,µg/m3,2022-01-01 06:00:00 +01:00,2022-01-01 07:00:00 +01:00,1,3
...,...,...,...,...,...,...,...,...,...,...,...
8731,BE,NET-Wallonia,BETN121,PM10,hour,0.0,µg/m3,2022-12-30 20:00:00 +01:00,2022-12-30 21:00:00 +01:00,1,3
8732,BE,NET-Wallonia,BETN121,PM10,hour,0.0,µg/m3,2022-12-30 21:00:00 +01:00,2022-12-30 22:00:00 +01:00,1,3
8733,BE,NET-Wallonia,BETN121,PM10,hour,0.0,µg/m3,2022-12-30 22:00:00 +01:00,2022-12-30 23:00:00 +01:00,1,3
8734,BE,NET-Wallonia,BETN121,PM10,hour,NaN,µg/m3,2022-12-30 23:00:00 +01:00,2022-12-31 00:00:00 +01:00,-1,3


In [6]:
# pollutant segmentation
pollutants_list = df_bg_allpollutants.AirPollutant.unique()
pollutants_code = [14, 8, 9, 6, 1, 10]
polutants_urban_dict = dict(zip(pollutants_list, pollutants_code ))

In [7]:
#DataFrames per pollutant
polutants_urban_dict
df_bg_co = df_bg_allpollutants.query("AirPollutant == 'CO'").reset_index().drop(columns = 'index')
df_bg_no2 = df_bg_allpollutants.query("AirPollutant == 'NO2'").reset_index().drop(columns = 'index')
df_bg_o3 = df_bg_allpollutants.query("AirPollutant == 'O3'").reset_index().drop(columns = 'index')
df_bg_pm10 = df_bg_allpollutants.query("AirPollutant == 'PM10'").reset_index().drop(columns = 'index')
df_bg_pm25 = df_bg_allpollutants.query("AirPollutant == 'PM2.5'").reset_index().drop(columns = 'index')

#dict with DataFrames
dfs_all_pollutants = {'Co':df_bg_co, 'No2':df_bg_no2, 'PM10':df_bg_pm10, 'PM25':df_bg_pm25, 'o3':df_bg_o3}

In [8]:
#Verification of stations no. by pollutant
[print(x,':',len(y.AirQualityStationEoICode.value_counts()), 'stations') for x, y in dfs_all_pollutants.items()];

Co : 21 stations
No2 : 87 stations
PM10 : 71 stations
PM25 : 73 stations
o3 : 41 stations


## Statistics obtention
(package usage)

In [9]:
df_co_bg_valids = airqstats.get_valid_measurements(df_bg_co, 2)

**Mean**

In [10]:
airqstats.get_mean(df_co_bg_valids, 'AirQualityStationEoICode', 'Concentration')

,stations,mean_valid_year
0,BETR701,0.230269
1,BETR202,0.252340
2,BETB001,0.299177
3,BETR002,0.224857
4,BETR223,0.230245
5,BETR001,0.245163
6,BETB004,0.293317
7,BETN132,0.176444
8,BETR512,0.214725
9,BETB008,0.450312


**Maximum**

In [11]:
airqstats.get_max(df_co_bg_valids, 'AirQualityStationEoICode', 'Concentration')

,stations,max_valid
0,BETR701,1.765
1,BETR202,1.310
2,BETB001,2.835
3,BETR002,1.205
4,BETR223,2.470
5,BETR001,1.830
6,BETB004,1.870
7,BETN132,0.910
8,BETR512,1.520
9,BETB008,43.685


**Percentile 99th**

In [12]:
airqstats.percentile_99(df_co_bg_valids, 'AirQualityStationEoICode', 'Concentration')

,stations,quant99_valid
0,BETR701,0.77500
1,BETR202,0.68000
2,BETB001,0.83330
3,BETR002,0.58000
4,BETR223,0.80440
5,BETR001,0.95110
6,BETB004,1.02500
7,BETN132,0.40500
8,BETR512,0.77000
9,BETB008,1.36020


## Madrid's data
This was obtained from Madrid's city council webpage.

In [25]:
# paths
path_single = 'Data/Madrid/Madrid_hourly_pollutants_2022/abr_mo22.csv'
path_all = 'Data/Madrid/Madrid_hourly_pollutants_2022' 

In [26]:
# inspection of single file to determine columns to import
test_md = pd.read_csv(path_single, sep = ';')
test_md

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2022,4,1,2.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,1.00,V
1,28,79,4,1,28079004_1_38,2022,4,2,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,1.00,V
2,28,79,4,1,28079004_1_38,2022,4,3,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,1.00,V
3,28,79,4,1,28079004_1_38,2022,4,4,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,2.00,V
4,28,79,4,1,28079004_1_38,2022,4,5,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,2.00,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3946,28,79,60,14,28079060_14_6,2022,4,26,50.09,V,...,63.49,V,65.09,V,45.16,V,45.42,V,61.38,V
3947,28,79,60,14,28079060_14_6,2022,4,27,51.00,V,...,75.24,V,48.85,V,37.84,V,26.06,V,25.09,V
3948,28,79,60,14,28079060_14_6,2022,4,28,7.72,V,...,81.44,V,74.65,V,30.52,V,32.55,V,60.62,V
3949,28,79,60,14,28079060_14_6,2022,4,29,70.44,V,...,98.62,V,90.81,V,56.54,V,80.17,V,80.78,V


### Data assessment

##### Wrangling required for importing
1. **Importing**  
Separator is ';'.  
2. **Column selection**  
_Province and Municipio_ are not relevant for this study for now. We need to consider if nominal information of the location is neccesary; the same applies to _PUNTO_MUESTREO_.  
_Magnitud_ is neccesary as this is the column that contains the pollutants.  
3. **Data Preparation**  
Each pollutant had their own df; looping is done using stations grouping them yearly.

Data preparation

In [16]:
# pollutants
os.path.exists(path_all)
files = glob.glob((os.path.join(path_all, "*.csv")))
df_madrid_allpollutants = pd.concat([pd.read_csv(fp, sep=';').drop(columns = ['PROVINCIA','MUNICIPIO'])
       for fp in files])
df_madrid_allpollutants

,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,4,1,28079004_1_38,2022,4,1,2.00,V,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,1.00,V
1,4,1,28079004_1_38,2022,4,2,1.00,V,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,1.00,V
2,4,1,28079004_1_38,2022,4,3,1.00,V,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,1.00,V
3,4,1,28079004_1_38,2022,4,4,1.00,V,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,2.00,V
4,4,1,28079004_1_38,2022,4,5,1.00,V,1.00,V,...,2.00,V,2.00,V,2.00,V,2.00,V,2.00,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4087,60,14,28079060_14_6,2022,10,27,28.22,V,33.60,V,...,5.11,V,2.32,V,7.45,V,3.83,V,5.15,V
4088,60,14,28079060_14_6,2022,10,28,13.70,V,8.68,V,...,25.20,V,36.18,V,23.32,V,4.00,V,8.90,V
4089,60,14,28079060_14_6,2022,10,29,14.76,V,24.90,V,...,64.86,V,48.46,V,51.21,V,47.40,V,44.05,V
4090,60,14,28079060_14_6,2022,10,30,32.58,V,42.38,V,...,15.22,V,15.42,V,11.50,V,17.12,V,24.05,V


In [18]:
#DataFrames por specific pollutant
pollutants_list = df_madrid_allpollutants.MAGNITUD.unique()
polutants_urban_dict = {'co' : 6, 'no' : 7, 'no2' : 8, 'pm25' : 9, 'pm10' : 10, 'nox' : 12}

df_madrid_co = df_madrid_allpollutants.query("MAGNITUD == @polutants_urban_dict['co']").reset_index().drop(columns = 'index')
df_madrid_no = df_madrid_allpollutants.query("MAGNITUD == @polutants_urban_dict['no']").reset_index().drop(columns = 'index')
df_madrid_no2 = df_madrid_allpollutants.query("MAGNITUD == @polutants_urban_dict['no2']").reset_index().drop(columns = 'index')
df_madrid_nox = df_madrid_allpollutants.query("MAGNITUD == @polutants_urban_dict['nox']").reset_index().drop(columns = 'index')
df_madrid_pm10 = df_madrid_allpollutants.query("MAGNITUD == @polutants_urban_dict['pm10']").reset_index().drop(columns = 'index')
df_madrid_pm25 = df_madrid_allpollutants.query("MAGNITUD == @polutants_urban_dict['pm25']").reset_index().drop(columns = 'index')

#list with data frames
list_all_pollutants = {'Co2':df_madrid_co, 'No':df_madrid_no, 'No2':df_madrid_no2, 'PM10':df_madrid_pm10, 'PM25':df_madrid_pm25, 'Nox':df_madrid_nox}

## Statistics obtention
(package usage)

In [19]:
airqstats.get_valid_measurements(df_madrid_co, 1)

,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,...,H21,V21,H22,V22,H23,V23,H24,V24,valid_lectures,unvalid_lectures
0,4,6,28079004_6_48,2022,4,1,0.2,V,0.2,V,...,0.3,V,0.3,V,0.3,V,0.3,V,"[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ...",[]
1,4,6,28079004_6_48,2022,4,2,0.2,V,0.2,V,...,0.3,V,0.3,V,0.3,V,0.3,V,"[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ...",[]
2,4,6,28079004_6_48,2022,4,3,0.3,V,0.2,V,...,0.3,V,0.3,V,0.3,V,0.2,V,"[0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.5, ...",[]
3,4,6,28079004_6_48,2022,4,4,0.2,V,0.2,V,...,0.3,V,0.3,V,0.3,V,0.2,V,"[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1]"
4,4,6,28079004_6_48,2022,4,5,0.2,V,0.2,V,...,0.2,V,0.2,V,0.2,V,0.2,V,"[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, ...","[-1, -1, -1, -1, -1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,56,6,28079056_6_48,2022,10,27,0.4,V,0.5,V,...,0.7,V,0.9,V,0.8,V,0.4,V,"[0.4, 0.5, 0.3, 0.4, 0.2, 0.2, 0.4, 0.5, 0.6, ...",[]
1367,56,6,28079056_6_48,2022,10,28,0.4,V,0.3,V,...,0.7,V,0.8,V,0.8,V,0.9,V,"[0.4, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.6, ...",[]
1368,56,6,28079056_6_48,2022,10,29,1.3,V,1.0,V,...,0.7,V,0.6,V,0.8,V,0.7,V,"[1.3, 1.0, 0.5, 0.3, 0.3, 0.3, 0.3, 0.4, 0.6, ...",[]
1369,56,6,28079056_6_48,2022,10,30,0.5,V,0.5,V,...,0.7,V,0.7,V,0.5,V,0.4,V,"[0.5, 0.5, 0.4, 0.5, 0.3, 0.3, 0.3, 0.3, 0.3, ...",[]


**Mean**

In [21]:
airqstats.get_mean(df_madrid_co, 'ESTACION', 'valid_lectures')

,stations,mean_valid_year
0,4,0.217163
1,8,0.288433
2,35,0.434881
3,56,0.267977


**Maximum**

In [22]:
airqstats.get_max(df_madrid_co, 'ESTACION', 'valid_lectures')

,stations,max_valid
0,4,2.4
1,8,2.1
2,35,1.8
3,56,2.0


**Percentile 99th**

In [23]:
airqstats.percentile_99(df_madrid_co, 'ESTACION', 'valid_lectures')

,stations,quant99_valid
0,4,0.7
1,8,0.9
2,35,1.3
3,56,0.9
